# CiRA Demonstration

This notebook demonstrates the usage of the CiRA pipeline. Be sure to fulfill all prerequisites as specified in the [readme](README.md), especially the availability of the two pretrained models, both of which are [available for download on Zenodo](https://doi.org/10.5281/zenodo.7186287). If you run the code from the provided container, the models are already included.

## Locate the Models

The location of the two pre-trained models must be available to the CiRA converter. You can hardcode the location of the pre-trained models to the two variables `model_classification` and `model_labeling`, or use the subsequent snippet, in which the location of the models is determined. If you run the code locally, 

1. create a `.env` file and 
2. define the variables `MODEL_CLASSIFICATION` and `MODEL_LABELING` with the respective URLs. 

In [1]:
import model_locator

model_classification = model_locator.classification()
model_labeling = model_locator.labeling()

print(f'Classification model path: {model_classification}')
print(f'Labeling model path: {model_labeling}')

Classification model path: ./model/cira-classifier.bin
Labeling model path: ./model/cira-labeler.ckpt


## Create the Converter

Instantiate the CiRA converter which is realized in [src.cira.CiRAConverter](./src/cira.py). The instantiation of the converter will output some warning messages, which can be safely ignored.

In [2]:
from src.cira import CiRAConverter
cira = CiRAConverter(classifier_causal_model_path=model_classification, converter_s2l_model_path=model_labeling)

c:\Users\juf\Workspace\BTH\NLP_RE\cira\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exac

## Apply the converter

Finally, apply the converter to utilize the CiRA pipeline. This usually includes four steps:

1. Define a single natural language sentence. CiRA is currently only trained on grammatically correct, full sentences.
2. Classify the sentence to determine, whether the sentence is causal or nor.
3. If the sentence is causal, process it to generate labels, convert it into a cause-effect-graph, and finally derive a minimal set of test cases covering the requirement.

Non-causal sentences cannot be processed by the pipeline.

In [21]:
sentence: str = "Upon deploying the system a log has to be created."

In [22]:
causal, confidence = cira.classify(sentence)
print(f'CiRA classified the sentence "{sentence}" to be {"*" if causal else "*non"}causal* with a confidence of {confidence:.2%}.')

CiRA classified the sentence "Upon deploying the system a log has to be created." to be *causal* with a confidence of 99.80%.


In [23]:
if causal:
    labels, graph, suite = cira.process(sentence)

    print(f'Labels associated to the sentence: {labels}\n')
    print(f'Converted cause-effect-graph: {graph}\n')
    print(f'Minimal test suite: \n{suite}')

Labels associated to the sentence: [[5> (L0) Cause1  (None L1): [15> (L2) Variable <25] [5> (L4) Condition <14] <25], [26> (L1) Effect1 : [26> (L3) Variable <31] [32> (L5) Condition <49] <49], [15> (L2) Variable <25], [26> (L3) Variable <31], [5> (L4) Condition <14], [32> (L5) Condition <49]]

Converted cause-effect-graph: [the system].(deploying) ===> [a log].(has to be created)

Minimal test suite: 
  id  | the system     | a log
----  ---------------  -----------------------
   1  | deploying      | has to be created
   2  | not deploying  | not has to be created
